In [90]:
import os
import random
import hashlib
from sage.arith.power import generic_power


In [106]:
class KEMRSA():
    def __init__(self, param):
        #parametro de segurança
        self.p = 0
        self.q = 0
        self.n = 0
        self.e = 0
        self.d = 0
        self.to_key = None
        self.size = param
        self.chave_cifrada = None
        self.chave_decifrada = None
        self.chave_gerada_e = None
        self.chave_gerada_d = None
        self.salt = 0

    def generate_keys(self):

        #temos de encontrar p e q primos com size/2 bits
        while not (self.p in Primes()):
            self.p = next_prime(ZZ.random_element(pow(2,self.size/2-1)+1, pow(2,self.size/2)-1))

        while not (self.q in Primes()):
            self.q = next_prime(ZZ.random_element(pow(2,self.size/2-1)+1, pow(2,self.size/2)-1))

        #calcular o n 
        self.n = self.p * self.q
        
        #calcular o phi
        phi = (self.p -1) * (self.q-1)

        #e is a pseudo-random integer
        self.e = ZZ.random_element(phi)

        while (gcd(e,phi)!=1):
            self.e = ZZ.random_element(phi)

        #we use the extended Euclidean algorithm to calculate d 
        bezout = xgcd(self.e,phi)
        self.d = Integer(mod(bezout[1], phi))

        if(mod(self.d*self.e, phi) == 1):
            print("Congruente")
        else: 
            print("Não Congruente")
            self.generate_keys()  

        #Assim temos todos os parâmetros para as chaves privadas e públicas 
        #public = (n,e)
        #privada = (p,q,d)

    def encapsulamento(self, chave):
        self.to_key = int(chave)
        #Gera o salt
        self.salt = os.urandom(16)
        
        key_to_bytes = self.to_key.to_bytes(int(self.size/8), "big")
        self.chave_gerada_e = hashlib.pbkdf2_hmac('sha256', key_to_bytes, self.salt, 100000)

        self.cifrar()
        print("Mensagem cifrada" , self.chave_cifrada)

    def cifrar(self):
        self.chave_cifrada = power_mod(self.to_key, self.e, self.n)

    def decifrar(self):
        key_to_dec = int(self.chave_cifrada)
        self.chave_decifrada = power_mod(key_to_dec, self.d, self.n)
    
    def revelar(self):
        self.decifrar()

        key_to_bytes = int(self.chave_decifrada).to_bytes(int(self.size/8), "big")
        self.chave_gerada_d = hashlib.pbkdf2_hmac('sha256', key_to_bytes, self.salt, 100000)
        print("Mensagem decifrada" , self.chave_decifrada)

    def verificar(self):
        if (self.chave_gerada_e == self.chave_gerada_d):
            print("As chaves coincidem!")
        else: 
            print("As chaves não coincidem!")

In [107]:
kem = KEMRSA(1024)
kem.generate_keys()
elem =ZZ.random_element(1024)
print(elem)
kem.encapsulamento(elem)
kem.revelar()
kem.verificar()

Não Congruente
Congruente
313
Mensagem cifrada 20667359703789079298494833492705483148114092544793671772476340888937221785812998117321779339618175111024681189133449947209820226281262897152174905454514982924449545353261679912150792941682139078895291001096154042350797717292662177213084944612114528105307976828307546973058689549999551782253914317627752036223
Mensagem decifrada 313
As chaves coincidem!
